# Recommender System
## Group 7

This shows some attempts to build a recommender system

In [1]:
# import libraries
import graphlab as gl
import numpy as np
import pandas as pd

## Input data : keep a dataframe containing user-item interactions

In [2]:
# create SFrame for Amsterdam ins data
amsterdam_sf = gl.SFrame.read_csv('../DataSources/FourCityDataset/instagram/amsterdam.txt',
                        delimiter=' ',header=False)
amsterdam_sf.rename({'X1':'user_id','X2':'venue_id','X3':'#visits(normalized)'})

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489589550.log


This non-commercial license of GraphLab Create for academic use is assigned to r.ramoschavez@student.tudelft.nl and will expire on February 23, 2018.


Finished parsing file /home/roy/Documents/python_workspace/IR/RecommendationSystem/IR_RecommendationHE/DataSources/FourCityDataset/instagram/amsterdam.txt

Parsing completed. Parsed 100 lines in 0.046839 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/roy/Documents/python_workspace/IR/RecommendationSystem/IR_RecommendationHE/DataSources/FourCityDataset/instagram/amsterdam.txt

Parsing completed. Parsed 25442 lines in 0.034274 secs.

user_id,venue_id,#visits(normalized)
7546845,4d9e0491baae54817c8aff64,0.666666666667
574349047,4aa8b27bf964a520b65120e3,0.833333333333
263031464,4f91796ee4b0d5d7897f3a13,0.666666666667
272238078,4c6bd9fb9669e21e7acdab51,0.666666666667
476498302,4bd6befa6798ef3bba83658d,0.666666666667
616834777,4a27083ff964a52013901fe3,0.666666666667
209919551,4aa68d04f964a520274a20e3,0.75
323561614,4a270748f964a5207b8d1fe3,0.666666666667
250275530,514e0cbce4b0679548f4d62a,0.833333333333
29782417,4a27009ff964a520e4831fe3,0.75


In [3]:
amsterdam_sf.head(5)

user_id,venue_id,#visits(normalized)
7546845,4d9e0491baae54817c8aff64,0.666666666667
574349047,4aa8b27bf964a520b65120e3,0.833333333333
263031464,4f91796ee4b0d5d7897f3a13,0.666666666667
272238078,4c6bd9fb9669e21e7acdab51,0.666666666667
476498302,4bd6befa6798ef3bba83658d,0.666666666667


## Keep another list for iem metadata
todo: enrich the metadata from foursquare

In [4]:
#items = gl.SFrame.read_csv(...)

## Create a simple model
1. Random split data into training data and validation data
2. We have a target column: #visits, thus we can create a model to predict which new places a user would visit frequently.

In [5]:
# split data into training and validation
training_data, validation_data = gl.recommender.util.random_split_by_user(amsterdam_sf, 'user_id', 'venue_id')
ranking_factorization_model = gl.recommender.create(training_data, 'user_id', 'venue_id',target='#visits(normalized)')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 24266 observations with 4317 users and 5617 items.

Data prepared in: 0.066901s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.00476901

Final training RMSE: 0.052694

## Use the model created to recommend places to a user

In [6]:
# first select a user, and see where he/she has visited
mask = amsterdam_sf['user_id']==7546845
amsterdam_sf[mask]

user_id,venue_id,#visits(normalized)
7546845,4d9e0491baae54817c8aff64,0.666666666667
7546845,4a995531f964a520da2d20e3,0.666666666667
7546845,4de6588a18389f0558772bbf,0.666666666667
7546845,4c06841f191f20a128a2e414,0.666666666667
7546845,4a27079bf964a5205a8e1fe3,0.666666666667
7546845,4a2705e6f964a5204a891fe3,0.75
7546845,4f707285e4b03b575ed77f23,0.666666666667


In [7]:
# recommend top3 places for this user
ranking_factorization_model.recommend(users=['7546845'],k=3)

user_id,venue_id,score,rank
7546845,4b448a5ef964a52047f625e3,0.748234120653,1
7546845,4b5fe77bf964a5203fd029e3,0.742264886843,2
7546845,4a270702f964a520ac8c1fe3,0.730448016421,3


In [9]:
graphlab.canvas.set_target('ipynb')
view = ranking_factorization_model.views.overview(validation_set=validation_data)
view.show()

NameError: name 'graphlab' is not defined

## create a popularity model as our baseline
This model just recommend based on popularity, no personalization

In [10]:
import graphlab
popularity_model = graphlab.popularity_recommender.create(training_data,
                                                         user_id='user_id',
                                                         item_id='venue_id')

Recsys training: model = popularity

Warning: Column '#visits(normalized)' ignored.

To use this column as the target, set target = "#visits(normalized)" and use a method that allows the use of a target.

Preparing data set.

Data has 24266 observations with 4317 users and 5617 items.

Data prepared in: 0.076455s

24266 observations to process; with 5617 unique items.

In [12]:
model_performance = graphlab.compare(validation_data, [popularity_model, ranking_factorization_model], user_sample=0.8)
graphlab.show_comparison(model_performance,[popularity_model,ranking_factorization_model])

compare_models: using 524 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    | 0.0248091603053 | 0.0170801526718 |
|   2    | 0.0248091603053 | 0.0309796437659 |
|   3    | 0.0241730279898 | 0.0466284987277 |
|   4    | 0.0233778625954 |  0.058524173028 |
|   5    | 0.0225190839695 | 0.0698155216285 |
|   6    | 0.0203562340967 | 0.0738231552163 |
|   7    |  0.01826608506  | 0.0771628498728 |
|   8    | 0.0164599236641 | 0.0785941475827 |
|   9    | 0.0161153519932 | 0.0844783715013 |
|   10   | 0.0152671755725 | 0.0898854961832 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+

## As shown above, our model now performs badly....
### We should enrich our data and train a much better model!!